In [ ]:
using JuMP
using MadNLP
using ProgressMeter
using CairoMakie
import Plots.plot as pplot
import Plots.plot! as pplot!

Classical Lennard-Jones potential (see also https://en.wikipedia.org/wiki/Lennard-Jones_potential):
$$
    W_{x, x'}(r) := 4 \left[
        \left(\frac{|x - x'|}{r}\right)^{12}
        - \left(\frac{|x - x'|}{r}\right)^6
    \right].
$$

The optimization algorithm experiences too large numbers for the above potential.
In order to overcome this we use a slightly different Lennard-Jones type potential:
$$
    W_{x, x'}(r) := 4 \left[
        \left(\frac{|x - x'|}{r}\right)^{\color{magenta}4}
        - \left(\frac{|x - x'|}{r}\right)^{\color{magenta}2}
    \right].
$$

In [ ]:
# Lennard-jones of squared distances instead of distances
function lennard_jones(cur_dist_sq; pref_dist=1, pot_min=-1)
    # the factor 2 assures that the global
    # minimum is attained at cur_dist == pref_dist
    q = pref_dist^2/(2*cur_dist_sq)
    -4*pot_min*(
        q^2 - q
    )
end

In [ ]:
pot_min = -4
pref_dist = 1.5
xs = range(pref_dist/2, 10, length=1000)
f(r) = lennard_jones(r, pref_dist=pref_dist, pot_min=pot_min)
ys = f.(xs.^2)
pplot(xs, ys, ylims=(pot_min-0.5, 7), label=nothing)

In [ ]:
N = 30
max_stretch = 0.25
time_stretch = 2
time_pause = 0.25
time_horizon = time_stretch + time_pause
fps = 120
pref_dist = 1/(N-1)
pot_min = -1.
elastic_thresh = 2*pref_dist
search_rad = 1.
diss_coeff = 1.
alpha = 100
eps = pref_dist/10
animation_width = 800
file_name = "crack_evol"
with_plasticity = true
two_sided_pull = true

In [ ]:
function dirichlet_delta(t; max_stretch=1, time_stretch=1, time_pause=1.5)
    # max_stretch*sin(2*pi/time_horizon*t)
    if t <= time_stretch
        return max_stretch*t/time_stretch
    else
        return max_stretch
    end
end
g_right(t) = 1 + dirichlet_delta(t; max_stretch=max_stretch, time_stretch=time_stretch, time_pause=time_pause)
if two_sided_pull
    g_left(t) = -dirichlet_delta(t; max_stretch=max_stretch, time_stretch=time_stretch, time_pause=time_pause)
else
    g_left(t) = 0
end
xs = range(0, time_horizon, length=1000)
ys = g_right.(xs)
pplot(xs, ys, ylims=(0, 1 + max_stretch), label=nothing)

In [ ]:
soft_max(x, y) = (x*exp(alpha*x) + y*exp(alpha*y)) / 
    (exp(alpha*x) + exp(alpha*y))
xs = range(-2., 2., length=1000)
soft_ys = [soft_max(x, 0.) for x in xs]
ys = [max(x, 0.) for x in xs]
pplot(xs, soft_ys, label="soft_+", legend=:topleft)
pplot!(xs, ys, label="+")

In [ ]:
step = 2
minmove = Model(
    optimizer_with_attributes(
        MadNLP.Optimizer,
        "print_level" => MadNLP.WARN,
        "blas_num_threads" => 4
    )
)

@NLparameter(minmove, prev_y[i=1:N] == (i-1)/(N-1))
if with_plasticity
    @NLparameter(minmove, max_elong_sq[i=1:N-1] == 1/(N-1)^2)
end

@variable(
    minmove,
    value(prev_y[i]) - search_rad <= y[i=1:N] <= value(prev_y[i]) + search_rad,
    start=value(prev_y[i])
)

# Dirichlet condition on the right endpoint
@constraint(
    minmove, dirichlet_right,
    y[N] == g_right((step-1)/fps)
)
@constraint(
    minmove, dirichlet_left,
    y[1] == g_left((step-1)/fps)
)
# Regularization
@constraint(
    minmove, pos_det[i=1:N-1],
    y[i+1] - y[i] >= eps
)

@expression(minmove, dist_sq[i=1:N-1], (y[i+1] - y[i])^2)
# Explicitely write out the softmax
if with_plasticity
    @NLexpression(
        minmove, damaged_dist_sq[i=1:N-1],
        (dist_sq[i]*exp(alpha*dist_sq[i]) + max_elong_sq[i]*exp(alpha*max_elong_sq[i])) /
            (exp(alpha*dist_sq[i]) + exp(alpha*max_elong_sq[i]))
    )
else
    @NLexpression(minmove, damaged_dist_sq[i=1:N-1], dist_sq[i])
end
W(d) = lennard_jones(d, pref_dist=pref_dist, pot_min=pot_min)
register(minmove, :W, 1, W, autodiff = true)
@NLexpression(
    minmove, energy, 
    .5/(N-1)*sum(W(damaged_dist_sq[i]) for i in 1:N-1)
)
@NLexpression(
    minmove, dissipation,
    .5/N*sum((y[i] - prev_y[i])^2 for i in 1:N)
)
@NLobjective(minmove, Min, energy + diss_coeff*fps*dissipation)

minmove

In [ ]:
optimize!(minmove)

In [ ]:
xs = range(0, 1, length=N)
pplot(xs, value.(prev_y), label="prev_y", legend=:topleft)
pplot!(xs, value.(y), label="y")

In [ ]:
function get_energy(y)
    N = length(y)
    0.5/(N-1)*sum(W.((y[2:end]-y[1:end-1]).^2))
end
abs(get_energy(value.(y)) - value(energy))

In [ ]:
function get_dissipation(y, prev_y)
    N = length(y)
    0.5/N*sum((y - prev_y).^2)
end
abs(get_dissipation(value.(y), value.(prev_y)) - value(dissipation))

In [ ]:
function get_total(y, prev_y, fps, diss_coeff)
    get_energy(y) + fps*diss_coeff*get_dissipation(y, prev_y)
end
abs(get_total(value.(y), value.(prev_y), fps, diss_coeff) - objective_value(minmove))

In [ ]:
get_total(value.(y), value.(prev_y), fps, diss_coeff)

In [ ]:
y_comp_inst = [(i-1)/(N-1) for i in 1:N]
y_comp_inst[N] = g((step-1)*time_horizon/fps)
get_total(y_comp_inst, value.(prev_y), fps, diss_coeff)

In [ ]:
interv_len = g((step-1)*time_horizon/fps)
y_comp_hom = [(i-1)*interv_len/(N-1) for i in 1:N]
get_total(y_comp_hom, value.(prev_y), fps, diss_coeff)

In [ ]:
minmoves = [value.(prev_y), value.(y)]

In [ ]:
@showprogress "Computing minmoves..." for step in 3:fps*time_horizon
    # update maximal elongation
    yval = value.(y)
    comp_max_elong_sq = (yval[2:end] - yval[1:end-1]).^2
    new_max_elong_sq = max(value.(max_elong_sq), comp_max_elong_sq)
    set_value.(max_elong_sq, new_max_elong_sq)
    
    set_value.(prev_y, value.(y))
    
    set_normalized_rhs(dirichlet_right, g_right((step-1)/fps))
    set_normalized_rhs(dirichlet_left, g_left((step-1)/fps))
    
    optimize!(minmove)
    
    push!(minmoves, value.(y))
end

In [ ]:
function y_monotone(y)
    if length(y) < 3
        return false
    end
    d = y[2] - y[1]
    for i in 3:length(y)
        new_d = y[3] - y[2]
        if new_d < d
            return false
        end
        d = new_d
    end
    return true
end

In [ ]:
for (i, y) in enumerate(minmoves)
    if !y_monotone(y)
        println(i)
        break
    end
end

In [ ]:
function animate_steps(minmoves, file_name; fps=30, width=800)
    folder_path = "experiments"
    if !ispath(folder_path)
        mkdir(folder_path)
    end
    
    aspect = 4.5
    ax_aspect = 6
    height = width/aspect
    max_right = maximum([y[end] for y in minmoves])
    min_left = minimum([y[1] for y in minmoves])
    max_len = max_right - min_left
    ax_height = max_len/ax_aspect
    N = length(minmoves[1])
    
    fig = Figure(resolution=(width, 200))
    ax = Axis(fig[1, 1], limits=(min_left-.1, max_right+.1, -ax_height/2, ax_height/2), aspect=ax_aspect)
    
    y = zeros(N)
    
    record(fig, "experiments/$file_name", minmoves; framerate = fps) do (x)
        empty!(ax)
        scatter!(x, y, color=:blue, markersize=4)
    end
end

In [ ]:
animate_steps(minmoves, "$file_name.mp4", fps=60, width=animation_width)